# Forecasting with Sktime

In [2]:
import pandas as pd
import numpy as np
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series

In [3]:
data = pd.read_feather('../../data/2.3_data_with_occupancies.feather')
data

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,...,ADR_7d_avg,ADR_30d_avg,ADR_7d_std,ADR_30d_std,ADR_ewm_3,ADR_ewm_7,occupancy_at_arrival,occupancy_at_departure,occupancy_pct_at_arrival,occupancy_pct_at_departure
UUID,,,,,,,,,,,,,,,,,,,,,
b6bcab0c-31d0-44e4-b75f-8829827e31e9,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,41,41,0.115169,0.115169
dea773eb-7d03-4ddf-b0e6-e5767c04eb95,0,2,1.0,0,BB,IRL,Offline TA/TO,TA/TO,0,0,...,82.00,87.43,17.41,16.89,83.17,86.84,41,201,0.115169,0.564607
a8e9c8ff-7d66-4082-a139-bf0f226e4c3d,0,2,0.0,0,BB,IRL,Offline TA/TO,TA/TO,0,0,...,82.53,82.87,16.78,16.76,76.39,82.53,41,201,0.115169,0.564607
2da831f5-1f82-40cf-986a-2cee9ceb4b63,0,2,0.0,0,HB,GBR,Offline TA/TO,TA/TO,0,0,...,67.26,79.90,10.61,19.67,66.03,75.82,41,201,0.115169,0.564607
8d9af511-c665-4e8c-8cef-a8ffa8bcb6e8,0,3,0.0,0,BB,ESP,Offline TA/TO,TA/TO,0,0,...,69.21,75.73,13.34,14.19,74.19,77.45,41,172,0.115169,0.483146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12af6175-d1a3-4fb3-bb0a-f9798c746ab4,0,2,0.0,0,SC,FRA,Online TA,TA/TO,0,0,...,127.00,114.50,30.31,26.63,136.18,130.86,271,102,0.761236,0.286517
c99d4bd2-691a-4a59-8076-c8455bc407d9,0,2,0.0,0,SC,GBR,Online TA,TA/TO,0,0,...,134.11,130.86,49.18,33.55,152.92,138.32,271,154,0.761236,0.432584
ad7a2ded-b7cc-4116-b52d-c7f284f7ad66,0,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,...,171.53,181.15,39.00,60.50,165.02,170.90,271,154,0.761236,0.432584
